# MetaTrader 5 Stock Data Fetcher

This notebook connects to MetaTrader 5 to fetch stock price data across multiple timeframes and stores it in the database.

**Features:**
- Fetches data for 7 different timeframes (1min, 5min, 15min, 30min, 1h, 4h, daily)
- Checks database for existing data to avoid duplicates
- Handles proper timeframe mapping between MT5 and database
- Gets stock ID from stocks table when fetching data

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import time
import sys
import os

# Connect to MetaTrader 5
if not mt5.initialize():
    print("Initialize() failed, error code =", mt5.last_error())
    quit()

# Set symbol, timeframe, and date range
symbol = "XAUUSD"  # Gold
timeframe = mt5.TIMEFRAME_M1  # 5-minute data
utc_from = datetime(year=2025, month=5, day=9,hour=10,minute=0)  # Start date
utc_to = datetime.now()

# Download data
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)

# Convert to DataFrame
df = pd.DataFrame(rates)

# convert time in seconds to datetime
df['time'] = pd.to_datetime(df['time'], unit='s')
# convert time to UTC +3
df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('Europe/Moscow')
# make sure to set the index to time
df.set_index('time', inplace=True)
print(df)




                              open     high      low    close  tick_volume  \
time                                                                         
2025-05-09 10:00:00+03:00  3308.05  3309.98  3308.05  3309.77           75   
2025-05-09 10:01:00+03:00  3309.77  3310.21  3308.90  3308.90           75   
2025-05-09 10:02:00+03:00  3309.37  3311.43  3309.37  3311.43           73   
2025-05-09 10:03:00+03:00  3311.43  3312.21  3311.21  3311.72           83   
2025-05-09 10:04:00+03:00  3311.72  3314.43  3311.50  3314.41          105   
...                            ...      ...      ...      ...          ...   
2025-05-15 00:19:00+03:00  3177.51  3177.97  3177.30  3177.51           88   
2025-05-15 00:20:00+03:00  3177.51  3178.59  3177.04  3178.34           93   
2025-05-15 00:21:00+03:00  3178.32  3178.73  3177.63  3177.90           80   
2025-05-15 00:22:00+03:00  3177.73  3178.56  3177.62  3177.78           71   
2025-05-15 00:23:00+03:00  3177.77  3178.20  3177.13  3178.12   

In [ ]:
import sqlite3
import sys

# Connect to the SQLite database
db_path = 'c:/Users/yoonus/Documents/GitHub/Stock_AI_Predictor/Data/data.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Helper function to check if database connection is valid
def check_db_connection():
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
        tables = cursor.fetchall()
        if tables:
            print(f"Connected to database. Found {len(tables)} tables.")
            return True
        return False
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return False

# Check database connection
if not check_db_connection():
    print("Failed to connect to database. Please check the path.")
    sys.exit(1)


Number of symbols: 110


# Close MT5 Connection

In [ ]:
# Shutdown connection
mt5.shutdown()

# Binance API

In [ ]:
from binance.client import Client
import pandas as pd

# No API key needed for public data
client = Client()




# Get historical kline/candlestick data
klines = client.get_historical_klines("BTCUSDT", "1m", "1 Jan, 2020", "28 Jan, 2020")

# Convert to DataFrame
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
           'quote_asset_volume', 'num_trades', 'taker_buy_base', 'taker_buy_quote', 'ignore']
df = pd.DataFrame(klines, columns=columns)

# Convert timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
#print(df[['timestamp', 'open', 'high', 'low', 'close', 'volume']].head())


In [2]:
df

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_buy_base,taker_buy_quote,ignore
0,2020-01-01 00:00:00,7195.24000000,7196.25000000,7183.14000000,7186.68000000,51.64281200,1577836859999,371233.51835535,493,19.59823000,140888.41428273,0
1,2020-01-01 00:01:00,7187.67000000,7188.06000000,7182.20000000,7184.03000000,7.24814800,1577836919999,52080.12778780,135,2.03177200,14599.21192429,0
2,2020-01-01 00:02:00,7184.41000000,7184.71000000,7180.26000000,7182.43000000,11.68167700,1577836979999,83903.74163545,202,5.47924400,39357.08177646,0
3,2020-01-01 00:03:00,7183.83000000,7188.94000000,7182.49000000,7185.94000000,10.02539100,1577837039999,72033.22664851,136,3.29496600,23680.57192367,0
4,2020-01-01 00:04:00,7185.54000000,7185.54000000,7178.64000000,7179.78000000,14.91110500,1577837099999,107066.52182466,161,2.36903300,17012.01513816,0
...,...,...,...,...,...,...,...,...,...,...,...,...
38876,2020-01-27 23:56:00,8912.09000000,8920.00000000,8911.91000000,8912.73000000,15.39012400,1580169419999,137210.07466135,186,7.26598300,64780.68743508,0
38877,2020-01-27 23:57:00,8912.89000000,8912.89000000,8905.64000000,8906.90000000,11.41435200,1580169479999,101686.22355148,175,3.30732100,29463.99101292,0
38878,2020-01-27 23:58:00,8907.87000000,8913.51000000,8906.26000000,8910.48000000,16.59114900,1580169539999,147807.22704581,174,10.08588200,89859.21802832,0
38879,2020-01-27 23:59:00,8910.23000000,8912.08000000,8905.62000000,8907.57000000,15.09636100,1580169599999,134491.70658113,195,6.00982400,53542.86851003,0
